# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd

In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName('Predicting churn') \
    .getOrCreate()

In [3]:
spark.sparkContext.getConf().getAll()

[('spark.app.name', 'Predicting churn'),
 ('spark.driver.port', '45589'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', '1e893d8feb6f'),
 ('spark.app.id', 'local-1632330149744'),
 ('spark.ui.showConsoleProgress', 'true')]

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [4]:
path = 'mini_sparkify_event_data.json'
user_log = spark.read.json(path)

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [5]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [9]:
total_rows = user_log.count()
total_users = user_log.select('userId').dropDuplicates().count()

print('Total registers: {}'.format(total_rows))
print('Total of unique users: {}'.format(total_users))

Total registers: 286500
Total of unique users: 226


In [15]:
# Displaying some userIds to understand how it is a user log
user_log.select('userId').dropDuplicates().sort('userId').show()

+------+
|userId|
+------+
|      |
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
|100016|
|100017|
+------+
only showing top 20 rows



Keep in mind the empty userId “ “, this will need to be treated. For now, let’s display a user log for a specific user.

In [17]:
user_log.select(['userId', 'firstname', 'page', 'song', 'status']).where(user_log.userId == '100').collect()

[Row(userId='100', firstname='Colin', page='NextSong', song='Bring Me To Life', status=200),
 Row(userId='100', firstname='Colin', page='NextSong', song='Strut (1993 Digital Remaster)', status=200),
 Row(userId='100', firstname='Colin', page='Thumbs Down', song=None, status=307),
 Row(userId='100', firstname='Colin', page='NextSong', song='Pretty Little Angel', status=200),
 Row(userId='100', firstname='Colin', page='NextSong', song='City On Our Knees', status=200),
 Row(userId='100', firstname='Colin', page='NextSong', song='Monday Morning (LP Version)', status=200),
 Row(userId='100', firstname='Colin', page='NextSong', song='Your Place [Album Version]', status=200),
 Row(userId='100', firstname='Colin', page='NextSong', song='Shoulder Lean [Featuring T.I.] (Amended Album Version) (AKA Radio Version)', status=200),
 Row(userId='100', firstname='Colin', page='Add to Playlist', song=None, status=200),
 Row(userId='100', firstname='Colin', page='NextSong', song="It's Not Like Everyone\x

In [22]:
# Displaying all the possible pages/actions a user can see/do
user_log.select('page').dropDuplicates().sort('page').show(22)

+--------------------+
|                page|
+--------------------+
|               About|
|          Add Friend|
|     Add to Playlist|
|              Cancel|
|Cancellation Conf...|
|           Downgrade|
|               Error|
|                Help|
|                Home|
|               Login|
|              Logout|
|            NextSong|
|            Register|
|         Roll Advert|
|       Save Settings|
|            Settings|
|    Submit Downgrade|
| Submit Registration|
|      Submit Upgrade|
|         Thumbs Down|
|           Thumbs Up|
|             Upgrade|
+--------------------+



Let's clear the data set, dropping all records without userId and sessionId, since one identifies the user and the other is used by HTTP protocol to network communications.

In [33]:
valid_log = user_log.dropna(how='any', subset=['userId', 'sessionId'])
valid_log = valid_log.filter(valid_log['userId'] != '')
print('Total user before the cleaning: {}'.format(user_log.count()))
print('Total user after the cleaning: {}'.format(valid_log.count()))

Total user before the cleaning: 286500
Total user after the cleaning: 278154


In [23]:
# Let's find the userIds who submit downgrade
valid_log.select(['userId']).where(user_log.page == 'Submit Downgrade').collect()

[Row(userId='131'),
 Row(userId='38'),
 Row(userId='141'),
 Row(userId='30'),
 Row(userId='20'),
 Row(userId='12'),
 Row(userId='11'),
 Row(userId='61'),
 Row(userId='81'),
 Row(userId='85'),
 Row(userId='140'),
 Row(userId='35'),
 Row(userId='54'),
 Row(userId='77'),
 Row(userId='59'),
 Row(userId='103'),
 Row(userId='95'),
 Row(userId='74'),
 Row(userId='85'),
 Row(userId='9'),
 Row(userId='24'),
 Row(userId='39'),
 Row(userId='39'),
 Row(userId='100'),
 Row(userId='140'),
 Row(userId='35'),
 Row(userId='13'),
 Row(userId='13'),
 Row(userId='96'),
 Row(userId='25'),
 Row(userId='109'),
 Row(userId='49'),
 Row(userId='20'),
 Row(userId='140'),
 Row(userId='39'),
 Row(userId='131'),
 Row(userId='92'),
 Row(userId='61'),
 Row(userId='100018'),
 Row(userId='100015'),
 Row(userId='100004'),
 Row(userId='100025'),
 Row(userId='100012'),
 Row(userId='100009'),
 Row(userId='100004'),
 Row(userId='100008'),
 Row(userId='100016'),
 Row(userId='100018'),
 Row(userId='200003'),
 Row(userId='2000

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.